In [1]:
import pandas as pd 
import numpy as np
from collections import Counter
import os
import unidecode

In [2]:
os.chdir('./data')

In [3]:
ls

 Le volume dans le lecteur C s'appelle Windows
 Le num‚ro de s‚rie du volume est A2FD-C15B

 R‚pertoire de C:\Users\Mohamed\git\Elections\legislatives\ETL Legislative 2T\data

04/06/2017  19:37    <DIR>          .
04/06/2017  19:37    <DIR>          ..
04/06/2017  19:23           185ÿ019 cdsp_legi1997t2_circ.xlsx
04/06/2017  19:41           470ÿ016 cdsp_legi2002t2_circ.xls
04/06/2017  19:23           293ÿ376 cdsp_legi2007t2_circ.xls
04/06/2017  19:23           163ÿ234 cdsp_legi2012t2_circ.xlsx
04/06/2017  19:23        12ÿ499ÿ195 dataframe_elections.xlsx
04/06/2017  19:46           727ÿ955 dataset_legislative_2nd.csv
               6 fichier(s)       14ÿ338ÿ795 octets
               2 R‚p(s)  708ÿ500ÿ590ÿ592 octets libres


In [77]:
legi_1997 = pd.read_excel("cdsp_legi1997t2_circ.xlsx")
legi_2002 = pd.read_excel("cdsp_legi2002t2_circ.xls")
legi_2007 = pd.read_excel("cdsp_legi2007t2_circ.xls")
legi_2012 = pd.read_excel("cdsp_legi2012t2_circ.xlsx")

In [78]:
legi_1997.head()

,Code département,département,circonscription,Infos vainqueur,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,2 Etiquette liste,2 nuance,2 voix,2 Elu,3 Nom candidat,3 Prénom candidat,3 Etiquette liste,3 nuance,3 voix,3 Elu
0,1,AIN,1,AIN 1,5,NaN,GODIN,ANDRE,"ASSOCIATION PARTI SOCIALISTE, PARTI RADICAL SO...",PRG,...,RASSEMBLEMENT POUR LA REPUBLIQUE,RPR,25396.0,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1,AIN,2,AIN 2,11,NaN,GUICHON,LUCIEN,RASSEMBLEMENT POUR LA REPUBLIQUE,RPR,...,RASSEMBLEMENT POUR LA REPUBLIQUE,RPR,22961.0,O,CLAVEL,EDMOND,FRONT NATIONAL,FRN,9546.0,N
2,1,AIN,3,AIN 3,11,NaN,MILLON,CHARLES,MAJORITE PRESIDENTIELLE,UDF,...,MAJORITE PRESIDENTIELLE,UDF,25270.0,O,NaN,NaN,NaN,NaN,NaN,NaN
3,1,AIN,4,AIN 4,11,NaN,VOISIN,MICHEL,UNION POUR LA DEMOCRATIE FRANCAISE,UDF,...,UNION POUR LA DEMOCRATIE FRANCAISE,UDF,32294.0,O,NaN,NaN,NaN,NaN,NaN,NaN
4,2,AISNE,1,AISNE 1,5,NaN,DOSIERE,RENE,PARTI SOCIALISTE,SOC,...,RASSEMBLEMENT POUR LA REPUBLIQUE,RPR,20073.0,N,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
legi_tab = pd.read_excel("dataframe_elections.xlsx")
legi_tab.ix[(legi_tab["code"]=="79|1")&(legi_tab["an"]==1997)&(legi_tab["nom"]=="PERRIN GAILLARD"), "nom"] = "GAILLARD"
legi_tab.ix[(legi_tab["code"]=="6|2")&(legi_tab["an"]==2002)&(legi_tab["nom"]=="MARLAND-MILITELLO"), "nom"] = "DE MARLAND"
legi_tab.ix[(legi_tab["code"]=="6|2")&(legi_tab["an"]==2007)&(legi_tab["nom"]=="MARLAND-MILITELLO"), "nom"] = "DE MARLAND"

In [80]:
def func_elu(a, b, c):
    if np.isnan(a) == True:
        return 'N'
    voix = a
    voix_max = max([a,b,c])
    if a < voix_max :
        return 'N'
    else :
        return 'O'

In [81]:
# on adapte les columns sur legi_2012 pour qu'elle supporte notre fonction
legi_2012["1 Elu"] = legi_2012.apply(lambda x : func_elu(x['1 voix'], x['2 voix'], x['3 voix']), axis=1)
legi_2012["2 Elu"] = legi_2012.apply(lambda x : func_elu(x['2 voix'], x['1 voix'], x['3 voix']), axis=1)
legi_2012["3 Elu"] = legi_2012.apply(lambda x : func_elu(x['3 voix'], x['2 voix'], x['1 voix']), axis=1)

In [82]:
# exemple sur 97
# get name
def in_format(data, annee):
    name = pd.melt(data, id_vars=["Code département", "département",'circonscription','Inscrits', 'Exprimés'], 
            value_vars=["1 Nom candidat","2 Nom candidat","3 Nom candidat"], value_name="name")
    prenom = pd.melt(data,
            value_vars=["1 Prénom candidat","2 Prénom candidat","3 Prénom candidat"], value_name="first_name")
    value = pd.melt(data,
            value_vars=["1 voix","2 voix","3 voix"], value_name="voix")
    elu = pd.melt(data, 
            value_vars=["1 Elu","2 Elu","3 Elu"], value_name="elu")
    df = pd.concat([name, prenom, value, elu], axis=1)

    del df["variable"]
    df["score"] = df.apply(lambda x : x['voix']/ x['Exprimés'], axis=1)
    df["an"] = annee

    df.dropna(inplace=True) 
    print(df.shape)
    return df

In [83]:
legi_1997 = in_format(legi_1997, 1997)
legi_2002 = in_format(legi_2002, 2002)
legi_2007 = in_format(legi_2007, 2007)
legi_2012 = in_format(legi_2012, 2012)

(1197, 11)
(1045, 11)
(933, 11)
(1101, 11)


In [84]:
legi_2t = pd.concat([legi_1997, legi_2002, legi_2007, legi_2012], axis=0)
legi_2t["code"] = legi_2t.apply(lambda x: str(x["Code département"])+'|'+str(int(x["circonscription"])), axis=1)

In [85]:
legi_2t.shape

(4276, 12)

In [86]:
legi_2t.sort_values(["code","name","first_name"]).head(2)

,Code département,département,circonscription,Inscrits,Exprimés,name,first_name,voix,elu,score,an,code
32,10,AUBE,1.0,55574.0,30564.0,BRET,LINE,13252.0,N,0.433582,2002,10|1
23,10,AUBE,1.0,58061.0,32500.0,BRET,Line,12793.0,N,0.393631,2007,10|1


In [87]:
legi_2t["name"] =  legi_2t["name"].map(lambda x: unidecode.unidecode(x.strip()))
legi_2t["first_name"] =  legi_2t["first_name"].map(lambda x: unidecode.unidecode(x.strip()).upper())
legi_tab["prenom"] =  legi_tab["prenom"] .map(lambda x: unidecode.unidecode(x.strip()).upper())
legi_tab["nom"] =  legi_tab["nom"].map(lambda x: unidecode.unidecode(x.strip()))

In [88]:
df_legi = pd.merge(legi_tab, legi_2t,left_on=["nom", "code", "an"], 
                   right_on=["name", "code", "an"])
df_legi.shape

(4237, 85)

*On merge sur le code plutot que nom prénom car trop de différence entre les deux tables
en esperant qu'il n'y ait qu'un candidat avec le même nom par code en une date donée*

In [89]:
#legi_2t[legi_2t["name"]=="PEROL DUMONT"]

In [90]:
df_legi = pd.merge(legi_tab, legi_2t,how="right" ,left_on=["nom", "code", "an"],
                   right_on=["name", "code", "an"])

pb_name = df_legi.ix[~df_legi.nom.notnull(), "name"].unique().tolist()

In [91]:
legi_2t["name"] = legi_2t["name"].map(lambda x : x.replace(" ","-") if x in pb_name else x)

In [92]:
# On personalise sur les quelques cas 
legi_2t.ix[(legi_2t["name"]=="PERRIN-GAILLARD") & (legi_2t["an"]==1997), "name"] = "GAILLARD"
legi_2t.ix[(legi_2t["name"]=="PERRIN-GAILLARD") & (legi_2t["an"]==2002), "name"] = "GAILLARD"
legi_2t.ix[(legi_2t["name"]=="GAAA") & (legi_2t["an"]==2002), "name"] = "GAIA"
legi_2t.ix[(legi_2t["name"]=="BACHELOT--NARQUIN") & (legi_2t["an"]==2002), "name"] = "BACHELOT-NARQUIN"
legi_2t.ix[(legi_2t["name"]=="VILLAUMe") & (legi_2t["an"]==2007), "name"] = "VILLAUME"
legi_2t.ix[(legi_2t["name"]=="CHEVeNEMENT") & (legi_2t["an"]==2007), "name"] = "CHEVENEMENT"
legi_2t.ix[(legi_2t["name"]=="GUeGOT") & (legi_2t["an"]==2007), "name"] = "GUEGOT"
legi_2t.ix[(legi_2t["name"]=="DE-SAINT-SERNIN") & (legi_2t["an"]==1997), "name"] = "DE SAINT-SERNIN"
legi_2t.ix[(legi_2t["name"]=="SAINT-SERNIN-DE") & (legi_2t["an"]==2002), "name"] = "DE SAINT-SERNIN"
legi_2t.ix[(legi_2t["name"]=="de-LA-RAUDIERE") & (legi_2t["an"]==2007), "name"] = "DE LA RAUDIERE"   
legi_2t.ix[(legi_2t["name"]=="ROCCA-SERRA-DE"), "name"] = "DE ROCCA SERRA"
legi_2t.ix[(legi_2t["name"]=="DE-DAVID-BEAUREGARD") & (legi_2t["an"]==1997), "name"] = "DE DAVID BEAUREGARD"
legi_2t.ix[(legi_2t["name"]=="DAVID-BEAUREGARD-DE") & (legi_2t["an"]==2002), "name"] = "DE DAVID BEAUREGARD"

In [93]:
# liste des nom et first name qui merge mal sur le inner
df_legi = pd.merge(legi_tab, legi_2t,how="right" ,left_on=["nom", "code", "an"],
                   right_on=["name", "code", "an"])
pb_name_2 = df_legi.ix[~df_legi.nom.notnull(), "name"].unique().tolist()

In [94]:
# clean des DE
legi_2t["name"] = legi_2t["name"].map(lambda x : "DE "+x.split("-")[0] if x in pb_name_2 else x)

In [95]:
df_legi = pd.merge(legi_tab, legi_2t,how="right" ,left_on=["nom", "code", "an"],
                   right_on=["name", "code", "an"])
pb_name_3 = df_legi[~df_legi.nom.notnull()][["name","first_name","an", "code","circonscription","Code département"]]

In [96]:
pb_name_3 = pb_name_3.reset_index()
pb_name_3

,index,name,first_name,an,code,circonscription,Code département
0,4293,DE BOURDOULEIX,GILLES,2002,49|5,5.0,49
1,4294,DE VAUTRIN,CATHERINE,2002,51|2,2.0,51
2,4295,DE GANSOINAT,MARIE-THERESE,2007,57|2,2.0,57
3,4296,DE VAUTRIN,CATHERINE,2007,51|2,2.0,51
4,4297,DE BOUZIANE,KHEIRA,2012,21|3,3.0,21
5,4298,DE MOREL,PIERRE,2012,48|1,1.0,48


In [97]:
legi_2t.ix[(legi_2t["name"]=="DE MOREL") & (legi_2t["an"]==2012), "name"] = "MOREL A L'HUISSIER"
legi_2t.ix[(legi_2t["name"]=="DE BOUZIANE") & (legi_2t["an"]==2012), "name"] = "BOUZIANE-LAROUSSI"
legi_2t.ix[(legi_2t["name"]=="DE VAUTRIN") & (legi_2t["an"]==2007), "name"] = "VAUTRIN-PENNAFORTE"
legi_2t.ix[(legi_2t["name"]=="DE VAUTRIN") & (legi_2t["an"]==2002), "name"] = "VAUTRIN-PENNAFORTE"
legi_2t.ix[(legi_2t["name"]=="DE GANSOINAT") & (legi_2t["an"]==2007), "name"] = "GANSOINAT"
legi_2t.ix[(legi_2t["name"]=="DE BOURDOULEIX") & (legi_2t["an"]==2002), "name"] = "BOURDOULEIX"

=> tout les noms doivent merger maintenant y'a t'il des doublons

In [98]:
df_legi = pd.merge(legi_tab, legi_2t,left_on=["nom", "code", "an"], 
                   right_on=["name", "code", "an"])
df_legi.shape
# On ne doit avoit que 4276 lignes

(4299, 85)

*Il nous manque personne mais on a des doublons vu la dim*  
regarder doublon même année, meme code, et meme nom

In [99]:
test = df_legi.sort_values(["name", "code", "an"])
test["name_lag"] = test["name"].shift(1)
test["first_lag"] = test["first_name"].shift(1)
test["code_lag"] = test["code"].shift(1)
test["an_lag"] = test["an"].shift(1)

In [100]:
test = test[(test['name']==test["name_lag"])&(test['code']==test["code_lag"])&(test['an']==test["an_lag"])]
pb_name = test["name"].unique()

In [101]:
#On fait un merge classique sur une table qui ne continet pas pb_name
df_legi_1 = pd.merge(legi_tab[~legi_tab["nom"].isin(pb_name)],
                     legi_2t[~legi_2t["name"].isin(pb_name)],
                     left_on=["nom", "code", "an"], 
                     right_on=["name", "code", "an"])

#puis merge avec prenom sur pb_name
df_legi_2 = pd.merge(legi_tab[legi_tab["nom"].isin(pb_name)],
                     legi_2t[legi_2t["name"].isin(pb_name)],
                     left_on=["nom","prenom", "code", "an"], 
                     right_on=["name","first_name", "code", "an"])

In [102]:
print(df_legi_1.shape)
print(df_legi_2.shape)

(4221, 85)
(51, 85)


*Il manque 3 candidats*

In [103]:
df_legi_2 = pd.merge(legi_tab[legi_tab["nom"].isin(pb_name)],
                     legi_2t[legi_2t["name"].isin(pb_name)],
                     how="right",
                     left_on=["nom","prenom", "code", "an"], 
                     right_on=["name","first_name", "code", "an"])

df_legi_2[~df_legi_2.nom.notnull()][["name","first_name","an", "circonscription","Code département"]]

,name,first_name,an,circonscription,Code département
51,DE ROCCA SERRA,JEAN PAUL,1997,2.0,2A
52,BERNARD,JEAN LOUIS,1997,3.0,45
53,BERNARD,J. LOUIS,2002,3.0,45
54,BENARD,"ALAIN, ROBERT",2002,2.0,974


In [104]:
# On modifie manuellement
legi_2t.ix[(legi_2t["name"]=="DE ROCCA SERRA")&(legi_2t["first_name"]=="JEAN PAUL"), "first_name"] = "JEAN-PAUL"
legi_2t.ix[(legi_2t["name"]=="BERNARD")&(legi_2t["Code département"]==45),"first_name"] = "JEAN-LOUIS"
legi_2t.ix[(legi_2t["name"]=="BENARD")&(legi_2t["code"]=="974|2"),"first_name"] = "ALAIN"

In [105]:
#On fait un merge classique sur une table qui ne continet pas pb_name
df_legi_1 = pd.merge(legi_tab[~legi_tab["nom"].isin(pb_name)],
                     legi_2t[~legi_2t["name"].isin(pb_name)],
                     left_on=["nom", "code", "an"], 
                     right_on=["name", "code", "an"])

#puis merge avec prenom sur pb_name
df_legi_2 = pd.merge(legi_tab[legi_tab["nom"].isin(pb_name)],
                     legi_2t[legi_2t["name"].isin(pb_name)],
                     left_on=["nom","prenom", "code", "an"], 
                     right_on=["name","first_name", "code", "an"])

print(df_legi_1.shape)
print(df_legi_2.shape)
print(df_legi_1.shape[0]+df_legi_2.shape[0])

(4221, 85)
(55, 85)
4276


In [106]:
df_legi = pd.concat([df_legi_1, df_legi_2], axis=0)

In [107]:
df_legi.columns

Index(['an', 'c_dep', 'dep', 'circo', 'code', 'inscrits', 'geo_frontalier',
       'geo_dom', 'geo_idf', 'etrangers', 'part_impose', 'revenus_q1',
       'revenus_med', 'revenus_q3', 'ecart_revenus', 'chom_tot',
       'chom_tot_evol_5', 'chom_jeunes', 'chom_jeunes_evol_5', 'chom_adultes',
       'chom_adultes_evol_5', 'chom_seniors', 'chom_seniors_evol_5', 'p_agri',
       'p_commercants', 'p_cadres', 'p_intermed', 'p_employes', 'p_ouvriers',
       'd_brevet', 'd_bep', 'd_bac', 'd_sup', 'circo_parti', 'circo_nuance',
       'circo_nuance_groupe', 'circo_bloc', 'circo_leg_meme_nuance',
       'nb_candidats_meme_bloc', 'score_nuance_groupe_prec_leg',
       'score_bloc_prec_leg', 'taux_vote_leg', 'score_leg_exg', 'score_leg_g',
       'score_leg_c', 'score_leg_d', 'score_leg_exd', 'score_leg_div',
       'circo_nuance_groupe_pres', 'circo_pres_meme_nuance',
       'circo_meme_nuance_president', 'score_nuance_groupe_pres',
       'score_bloc_pres', 'taux_vote_pres', 'score_pres_exg', 's

# Tout lancer jusqu'ici puis choisir les colones ci-dessous

In [108]:
columns_to_keep =['an','Code département', 'département',
       'circonscription', 'code','name', 'first_name',"sexe",
        "circo_bloc","circo_leg_meme_nuance","circo_pres_meme_nuance",
        "score_bloc_pres", 'depute_sortant', 'ancien_depute',
       'au_gouvernement', 'ancien_ministre', "bloc", "p_voix",
         'voix_y','Inscrits', 'score','elu'
]

**creer:**  
* une variable d'oppostion selon nuance un peu commme la variable candidat du même bloc  
* ~~score du groupe politque au second tour dernière legislative~~
* ~~score du bloc politique au premier tour~~
* ~~score candidat au second tour de la précédente législative~~ 
* score candidat au second tour d'une précédente législative 
* ~~indicatrice pour le nombre d'adversaire politique par bloc~~ 
* ~~triangulaire~~ 

In [109]:
df_legi_f = df_legi[columns_to_keep]

In [110]:
df_legi_f.head()

,an,Code département,département,circonscription,code,name,first_name,sexe,circo_bloc,circo_leg_meme_nuance,...,depute_sortant,ancien_depute,au_gouvernement,ancien_ministre,bloc,p_voix,voix_y,Inscrits,score,elu
0,2012,1,AIN,1.0,1|1,BRETON,XAVIER,M,Droite,1.0,...,1.0,1.0,0.0,0.0,Droite,0.2241,24219.0,79072.0,0.515715,O
1,2012,1,AIN,1.0,1|1,DEBAT,JEAN-FRANCOIS,M,Droite,0.0,...,0.0,0.0,0.0,0.0,Gauche,0.2316,22743.0,79072.0,0.484285,N
2,2012,1,AIN,2.0,1|2,DE LA VERPILLIERE,CHARLES,M,Droite,1.0,...,1.0,1.0,0.0,0.0,Droite,0.2209,22327.0,86951.0,0.443146,O
3,2012,1,AIN,2.0,1|2,EYRAUD,OLIVIER,M,Droite,0.0,...,0.0,0.0,0.0,0.0,Exdroite,0.1252,8530.0,86951.0,0.169303,N
4,2012,1,AIN,2.0,1|2,RAYMOND,MICHEL,M,Droite,0.0,...,0.0,0.0,0.0,0.0,Gauche,0.1368,19526.0,86951.0,0.387551,N


* triangulaire

In [111]:
triangulaire = df_legi.groupby(["an","code"])["name"].count().reset_index()
triangulaire["top_triangulaire"] = triangulaire["name"].map(lambda x: 1 if x>2 else 0)
del triangulaire["name"]
df_legi_f = pd.merge(df_legi_f, triangulaire, how="left", on=["an","code"])

* lag second tour legislative

=> on réalise ça lors de la création de la table de score de second tour
comme cela on peut utiliser les prénoms normalisés

* indicatrice adversaire politique

In [112]:
test = pd.get_dummies(df_legi_f, columns=["bloc"], prefix="bloc_opposant")

In [113]:
test.columns

Index(['an', 'Code département', 'département', 'circonscription', 'code',
       'name', 'first_name', 'sexe', 'circo_bloc', 'circo_leg_meme_nuance',
       'circo_pres_meme_nuance', 'score_bloc_pres', 'depute_sortant',
       'ancien_depute', 'au_gouvernement', 'ancien_ministre', 'p_voix',
       'voix_y', 'Inscrits', 'score', 'elu', 'top_triangulaire',
       'bloc_opposant_Centre', 'bloc_opposant_Divers', 'bloc_opposant_Droite',
       'bloc_opposant_Exdroite', 'bloc_opposant_Exgauche',
       'bloc_opposant_Gauche'],
      dtype='object')

In [114]:
test = test.groupby(["code","an"])['bloc_opposant_Centre', 'bloc_opposant_Divers', 'bloc_opposant_Droite',
       'bloc_opposant_Exdroite', 'bloc_opposant_Exgauche',
       'bloc_opposant_Gauche'].sum().reset_index()

In [115]:
df_legi_f = pd.merge(df_legi_f, test, on=["code","an"])

In [116]:
df_legi_f["bloc"].unique()

array(['Droite', 'Gauche', 'Exdroite', 'Centre', 'Exgauche', 'Divers'], dtype=object)

In [117]:
df_legi_f.ix[df_legi_f["bloc"]=='Droite', 'bloc_opposant_Droite'] = df_legi_f.ix[df_legi_f["bloc"]=='Droite', 
                                    'bloc_opposant_Droite'].map(lambda x: x-1 if x>1 else 0)

df_legi_f.ix[df_legi_f["bloc"]=='Gauche', 'bloc_opposant_Gauche'] = df_legi_f.ix[df_legi_f["bloc"]=='Gauche', 
                                    'bloc_opposant_Gauche'].map(lambda x: x-1 if x>1 else 0)

df_legi_f.ix[df_legi_f["bloc"]=='Exdroite', 'bloc_opposant_Exdroite'] = df_legi_f.ix[df_legi_f["bloc"]=='Exdroite', 
                                    'bloc_opposant_Exdroite'].map(lambda x: x-1 if x>1 else 0)

df_legi_f.ix[df_legi_f["bloc"]=='Centre', 'bloc_opposant_Centre'] = df_legi_f.ix[df_legi_f["bloc"]=='Centre', 
                                    'bloc_opposant_Centre'].map(lambda x: x-1 if x>1 else 0)

df_legi_f.ix[df_legi_f["bloc"]=='Exgauche', 'bloc_opposant_Exgauche'] = df_legi_f.ix[df_legi_f["bloc"]=='Exgauche', 
                                    'bloc_opposant_Exgauche'].map(lambda x: x-1 if x>1 else 0)

df_legi_f.ix[df_legi_f["bloc"]=='Divers', 'bloc_opposant_Divers'] = df_legi_f.ix[df_legi_f["bloc"]=='Divers', 
                                    'bloc_opposant_Divers'].map(lambda x: x-1 if x>1 else 0)

In [118]:
df_legi_f.head()

,an,Code département,département,circonscription,code,name,first_name,sexe,circo_bloc,circo_leg_meme_nuance,...,Inscrits,score,elu,top_triangulaire,bloc_opposant_Centre,bloc_opposant_Divers,bloc_opposant_Droite,bloc_opposant_Exdroite,bloc_opposant_Exgauche,bloc_opposant_Gauche
0,2012,1,AIN,1.0,1|1,BRETON,XAVIER,M,Droite,1.0,...,79072.0,0.515715,O,0,0,0,0,0,0,1
1,2012,1,AIN,1.0,1|1,DEBAT,JEAN-FRANCOIS,M,Droite,0.0,...,79072.0,0.484285,N,0,0,0,1,0,0,0
2,2012,1,AIN,2.0,1|2,DE LA VERPILLIERE,CHARLES,M,Droite,1.0,...,86951.0,0.443146,O,1,0,0,0,1,0,1
3,2012,1,AIN,2.0,1|2,EYRAUD,OLIVIER,M,Droite,0.0,...,86951.0,0.169303,N,1,0,0,1,0,0,1
4,2012,1,AIN,2.0,1|2,RAYMOND,MICHEL,M,Droite,0.0,...,86951.0,0.387551,N,1,0,0,1,1,0,0


* score du bloc au premier tour

In [119]:
test = legi_tab.groupby(["code","an","bloc"])["p_voix"].sum().reset_index()
test = test.rename(columns={"p_voix":"voix_bloc_1nd"})

In [120]:
df_legi_f = pd.merge(df_legi_f, test, on=["code","an","bloc"])

In [121]:
df_legi_f.head()

,an,Code département,département,circonscription,code,name,first_name,sexe,circo_bloc,circo_leg_meme_nuance,...,score,elu,top_triangulaire,bloc_opposant_Centre,bloc_opposant_Divers,bloc_opposant_Droite,bloc_opposant_Exdroite,bloc_opposant_Exgauche,bloc_opposant_Gauche,voix_bloc_1nd
0,2012,1,AIN,1.0,1|1,BRETON,XAVIER,M,Droite,1.0,...,0.515715,O,0,0,0,0,0,0,1,0.2306
1,2012,1,AIN,1.0,1|1,DEBAT,JEAN-FRANCOIS,M,Droite,0.0,...,0.484285,N,0,0,0,1,0,0,0,0.2492
2,2012,1,AIN,2.0,1|2,DE LA VERPILLIERE,CHARLES,M,Droite,1.0,...,0.443146,O,1,0,0,0,1,0,1,0.2209
3,2012,1,AIN,2.0,1|2,EYRAUD,OLIVIER,M,Droite,0.0,...,0.169303,N,1,0,0,1,0,0,1,0.1252
4,2012,1,AIN,2.0,1|2,RAYMOND,MICHEL,M,Droite,0.0,...,0.387551,N,1,0,0,1,1,0,0,0.2024


* score bloc précédente législative 

In [122]:
last_lest = df_legi_f.groupby(["code","an","bloc"])["score"].sum().reset_index()
last_lest["an"] = last_lest["an"].map(lambda x: x+5)
last_lest= last_lest.rename(columns={"score":"p_voix_2nd_lag"})
last_lest.head()

,code,an,bloc,p_voix_2nd_lag
0,10|1,2002,Centre,0.438561
1,10|1,2002,Exdroite,0.171653
2,10|1,2002,Gauche,0.389786
3,10|1,2007,Droite,0.566418
4,10|1,2007,Gauche,0.433582


In [123]:
last_lest[last_lest.code=="1|2"]

,code,an,bloc,p_voix_2nd_lag
368,1|2,2002,Droite,0.439151
369,1|2,2002,Exdroite,0.182576
370,1|2,2002,Gauche,0.378273
371,1|2,2007,Droite,0.618939
372,1|2,2007,Gauche,0.381061
373,1|2,2017,Droite,0.443146
374,1|2,2017,Exdroite,0.169303
375,1|2,2017,Gauche,0.387551


In [124]:
df_legi_f = pd.merge(df_legi_f, last_lest, on=["code","an","bloc"], how="left")
df_legi_f.head()

,an,Code département,département,circonscription,code,name,first_name,sexe,circo_bloc,circo_leg_meme_nuance,...,elu,top_triangulaire,bloc_opposant_Centre,bloc_opposant_Divers,bloc_opposant_Droite,bloc_opposant_Exdroite,bloc_opposant_Exgauche,bloc_opposant_Gauche,voix_bloc_1nd,p_voix_2nd_lag
0,2012,1,AIN,1.0,1|1,BRETON,XAVIER,M,Droite,1.0,...,O,0,0,0,0,0,0,1,0.2306,0.536621
1,2012,1,AIN,1.0,1|1,DEBAT,JEAN-FRANCOIS,M,Droite,0.0,...,N,0,0,0,1,0,0,0,0.2492,0.463379
2,2012,1,AIN,2.0,1|2,DE LA VERPILLIERE,CHARLES,M,Droite,1.0,...,O,1,0,0,0,1,0,1,0.2209,NaN
3,2012,1,AIN,2.0,1|2,EYRAUD,OLIVIER,M,Droite,0.0,...,N,1,0,0,1,0,0,1,0.1252,NaN
4,2012,1,AIN,2.0,1|2,RAYMOND,MICHEL,M,Droite,0.0,...,N,1,0,0,1,1,0,0,0.2024,NaN


Certaine circonscription comme l'ain n'ont pas eu d'election en 2007...bizare  
last_lest[last_lest["code"]=="1|2"] 

* score candidat au second tour de la précédente législative

In [125]:
test = df_legi_f.groupby(["code","an","name"])["first_name"].count().reset_index()
test[test["first_name"]>1]

,code,an,name,first_name
723,2A|2,1997,DE ROCCA SERRA,2


In [126]:
test = df_legi_f[["code","an","name","first_name","score"]]
test["an"] = test["an"].map(lambda x: x+5)

C:\Users\Mohamed\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [127]:
test.index[[3271,3272]]
test = test.drop(test.index[[3271,3272]])

In [128]:
test[(test["name"]=="DE ROCCA SERRA")&(test["an"]==2002)]

,code,an,name,first_name,score


In [129]:
test = test.rename(columns={"score":"p_voix_candidat_2nd_lag"})
del test["first_name"]

In [130]:
df_legi_f = pd.merge(df_legi_f, test, on=["code","an","name"], how="left")

* score candidat au second tour d'une précédente législative 

In [131]:
# export
df_legi_f.to_csv("dataset_legislative_2nd.csv", index=False)

In [132]:
test = df_legi_f[(np.isnan(df_legi_f["p_voix_candidat_2nd_lag"])) & (df_legi_f["an"]!=1997)]

In [133]:
jj = df_legi_f.groupby(["code","an","bloc"])["score"].sum().reset_index()
jj[jj["code"]=="1|2"]

,code,an,bloc,score
368,1|2,1997,Droite,0.439151
369,1|2,1997,Exdroite,0.182576
370,1|2,1997,Gauche,0.378273
371,1|2,2002,Droite,0.618939
372,1|2,2002,Gauche,0.381061
373,1|2,2012,Droite,0.443146
374,1|2,2012,Exdroite,0.169303
375,1|2,2012,Gauche,0.387551


In [138]:
df_legi_f[["code", "an", "name", "first_name", "bloc", "score", "voix_bloc_1nd", "p_voix_2nd_lag", "p_voix_candidat_2nd_lag"]].ix[50:,]

,code,an,name,first_name,bloc,score,voix_bloc_1nd,p_voix_2nd_lag,p_voix_candidat_2nd_lag
50,7|2,2012,DUSSOPT,OLIVIER,Gauche,0.533514,0.2760,0.537134,0.537134
51,7|3,2012,BUIS,SABINE,Gauche,0.512308,0.2498,0.444094,NaN
52,7|3,2012,FLORY,JEAN-CLAUDE,Droite,0.487692,0.2460,0.555906,0.555906
53,8|1,2012,LEDOUX,CLAUDINE,Gauche,0.446900,0.2062,0.404770,0.404770
54,8|1,2012,POLETTI,BERENGERE,Droite,0.552856,0.2275,0.595230,0.595230
55,8|2,2012,LEONARD,CHRISTOPHE,Gauche,0.536282,0.2496,0.514802,NaN
56,8|2,2012,RAVIGNON,BORIS,Droite,0.463718,0.1601,0.485198,0.485198
57,9|2,2012,CALLEJA,PHILIPPE,Droite,0.329663,0.1458,0.389491,0.389491
58,9|2,2012,FAURE,ALAIN,Gauche,0.670337,0.2808,0.610509,NaN
59,10|1,2012,DHUICQ,NICOLAS,Droite,0.441824,0.2152,0.606369,0.606369


In [135]:
df_legi_f.groupby(["code","an","bloc"])["score"].sum().reset_index()

,code,an,bloc,score
0,10|1,1997,Centre,0.438561
1,10|1,1997,Exdroite,0.171653
2,10|1,1997,Gauche,0.389786
3,10|1,2002,Droite,0.566418
4,10|1,2002,Gauche,0.433582
5,10|1,2007,Droite,0.606369
6,10|1,2007,Gauche,0.393631
7,10|1,2012,Droite,0.441824
8,10|1,2012,Exdroite,0.238046
9,10|1,2012,Gauche,0.320130
